In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# IMPORT DEPENDENCIES

In [2]:
import matplotlib
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import zipfile
import cv2

import tensorflow as tf
from tensorflow.python.keras import Sequential
from tensorflow.keras import layers, optimizers
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras. layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, Dropout
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.initializers import glorot_uniform 
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint, LearningRateScheduler
from IPython.display import display
from tensorflow.keras import backend as K
from sklearn.preprocessing import StandardScaler, normalize
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score
from sklearn.manifold import TSNE
from sklearn.metrics.pairwise import cosine_similarity
import plotly.graph_objects as go
import plotly.express as px
%matplotlib inline


# DATASET

In [3]:
df = pd.read_csv("../input/sample-sales-data/sales_data_sample.csv", sep=",", encoding='Latin-1')

In [4]:
df.head()

In [5]:
df.dtypes

In [6]:
df['ORDERDATE'] = pd.to_datetime(df['ORDERDATE'])

In [7]:
df.info()

In [8]:
df.isna().sum()

In [9]:
df.columns

In [10]:
df.nunique()

In [11]:
df_drop = ['ORDERNUMBER','ADDRESSLINE2', 'CUSTOMERNAME', 'PHONE',
       'ADDRESSLINE1', 'CITY', 'STATE', 'POSTALCODE', 'TERRITORY', 'CONTACTLASTNAME', 'CONTACTFIRSTNAME',]
df = df.drop(df_drop, axis = 1)

In [12]:
df.head()

# EDA

In [13]:
exp_df = pd.read_csv("../input/sample-sales-data/sales_data_sample.csv", sep=",", encoding='Latin-1')
exp_df['ORDERDATE'] = pd.to_datetime(exp_df['ORDERDATE'])

In [14]:
def barplot_visualization(x):
    fig = plt.figure(figsize = (12,6))
    fig = px.bar(x = exp_df[x].value_counts().index, y = exp_df[x].value_counts(), color = exp_df[x].value_counts().index, height = 600)
    fig.show()

In [15]:
barplot_visualization('STATUS')

In [16]:
df.STATUS.value_counts()

In [17]:
df = df.drop('STATUS', axis = 1)

In [18]:
df.columns

In [19]:
barplot_visualization('PRODUCTLINE')

In [68]:
# barplot_visualization('PRODUCTLINE')

In [21]:
barplot_visualization('COUNTRY')

In [22]:
barplot_visualization('DEALSIZE')

In [23]:
def dummies(x):
    dummy = pd.get_dummies(df[x])
    df.drop(columns = x, inplace = True)
    return pd.concat([df,dummy], axis = 1)

In [24]:
exp_df.nunique()

In [25]:
exp_df.dtypes

In [26]:
df = dummies('PRODUCTLINE')

In [27]:
df = dummies('DEALSIZE')

In [28]:
df = dummies('COUNTRY')

In [29]:
df['PRODUCTCODE'] = pd.Categorical(df['PRODUCTCODE']).codes

In [30]:
df.PRODUCTCODE.describe()

In [31]:
df_group_date = df.groupby(by = 'ORDERDATE').sum()
df_group_date

In [32]:
fig = px.line(x = df_group_date.index, y = df_group_date.SALES, title = 'SALES')
fig.show()

In [33]:
df = df.drop('ORDERDATE', axis = 1)

In [34]:
df.MONTH_ID.value_counts()

In [35]:
barplot_visualization('MONTH_ID')

In [36]:
plt.figure(figsize= (10,10))
corr_matrix = df.iloc[:,:10].corr()
sns.heatmap(corr_matrix, annot = True, fmt = ".3f", cmap = 'PuBu', xticklabels = corr_matrix.columns.values, yticklabels = corr_matrix.columns.values)
plt.show()

In [37]:
df.drop('QTR_ID', axis = 1, inplace = True)
df.shape

In [38]:
plt.figure(figsize = (15,15))
fig = px.scatter_matrix(df,dimensions = df.columns[:8], color = 'MONTH_ID')

fig.update_layout(
    title = 'SALES',
    dragmode = 'select',
    width = 1100,
    height = 1100,
    hovermode = 'closest',
)
fig.show()

# CLUSTERING

In [39]:
sc = StandardScaler()
df_scaled = sc.fit_transform(df)

In [40]:
df_scaled

In [41]:
scores = []

for i in range(1,15):
    kmeans = KMeans(n_clusters = i)
    kmeans.fit(df_scaled)
    scores.append(kmeans.inertia_)
    
plt.plot(range(1,15),scores, 'bx-')
plt.xlabel('Clusters')
plt.ylabel('Scores')
plt.title('Best k value')
plt.show()

In [42]:
kmeans = KMeans(n_clusters = 5)
kmeans.fit(df_scaled)
labels = kmeans.labels_
y_kmeans = kmeans.fit_predict(df_scaled)

In [43]:
y_kmeans

In [44]:
labels

In [45]:
kmeans.cluster_centers_.shape

In [46]:
cluster_centers  = pd.DataFrame(data = kmeans.cluster_centers_, columns = [df.columns])
cluster_centers = sc.inverse_transform(cluster_centers)
cluster_centers  = pd.DataFrame(data = cluster_centers, columns = [df.columns])
cluster_centers

In [47]:
label_df = pd.DataFrame({'clusters' : labels})
df_cluster = pd.concat([df, label_df], axis = 1)
df_cluster.head()

In [48]:
df['ORDERLINENUMBER'] = df['ORDERLINENUMBER'].apply(lambda x: float(x))

In [49]:
for i in df.columns[:8]:
    plt.figure(figsize = (30,6))
    for j in range(5):
        plt.subplot(1,5,j+1)
        cluster = df_cluster[df_cluster['clusters'] == j]
        cluster[i].hist()
        plt.title(f'{i} \nCluster - {j}')

plt.show()
        
    

# PCA

In [50]:
pca = PCA(n_components = 2)
pca_done = pca.fit_transform(df_scaled)
pca_done.shape

In [51]:
df_pca = pd.DataFrame(data = pca_done, columns = ['pca1','pca2'])
df_pca = pd.concat([df_pca, label_df], axis = 1)
df_pca

In [52]:
plt.figure(figsize = (10,10))
ax = sns.scatterplot(x= 'pca1', y= 'pca2', hue = 'clusters', data = df_pca, palette = ['red','green','blue','black','yellow'])
plt.show()

In [53]:
pca = PCA(n_components = 3)
pca_done = pca.fit_transform(df_scaled)
pca_done.shape

In [54]:
df_pca = pd.DataFrame(data = pca_done, columns = ['pca1','pca2','pca3'])

In [55]:
df_pca = pd.concat([df_pca, label_df], axis = 1)

In [56]:
df_pca

In [57]:
fig = px.scatter_3d(df_pca, x = 'pca1', y = 'pca2', z = 'pca3', 
                   color = 'clusters', symbol = 'clusters', size_max = 18, opacity = 0.7)

fig.update_layout(margin = dict(l=0, r=0, b=0, t=0))

# AUTOENCODERS

In [58]:
from tensorflow.keras.optimizers import SGD

input_df = Input(shape = (37,))
x = Dense(128, activation = 'relu')(input_df)
x = Dense(512, activation = 'relu')(x)
x = Dense(1024, activation = 'relu')(x)
encoded = Dense(8, activation = 'relu')(x)
x = Dense(1024, activation = 'relu')(encoded)
x = Dense(512, activation = 'relu')(x)
x = Dense(128, activation = 'relu')(x)
decoded = Dense(37, activation = 'relu')(x) 

autoencoder = Model(input_df, decoded)
encoder = Model(input_df, encoded)

autoencoder.compile(optimizer = 'adam', loss = 'mean_squared_error')



In [59]:
autoencoder.fit(df, df, batch_size = 64, epochs = 200, verbose = 1)

In [60]:
autoencoder.save_weights('autoencoder_1.h5')

In [61]:
pred = encoder.predict(df_scaled)

In [62]:
scores = []

for i in range(1,15):
    kmeans = KMeans(n_clusters = i)
    kmeans.fit(pred)
    scores.append(kmeans.inertia_)
    
plt.plot(range(1,15),scores, 'bx-')
plt.xlabel('Clusters')
plt.ylabel('Scores')
plt.title('Best k value')
plt.show()

In [63]:
kmeans = KMeans(n_clusters = 5)
kmeans.fit(pred)
labels = kmeans.labels_
y_kmeans = kmeans.fit_predict(df_scaled)

In [64]:
labels

In [65]:
cluster_centers  = pd.DataFrame(data = kmeans.cluster_centers_, columns = [df.columns])
cluster_centers = sc.inverse_transform(cluster_centers)
cluster_centers  = pd.DataFrame(data = cluster_centers, columns = [df.columns])
cluster_centers

In [66]:
label_df = pd.DataFrame({'clusters' : labels})
df_cluster = pd.concat([df, label_df], axis = 1)
df_cluster.head()

In [67]:
for i in df.columns[:8]:
    plt.figure(figsize = (30,6))
    for j in range(5):
        plt.subplot(1,5,j+1)
        cluster = df_cluster[df_cluster['clusters'] == j]
        cluster[i].hist()
        plt.title(f'{i} \nCluster - {j}')

plt.show()
        
    